<a href="https://colab.research.google.com/github/Yann-Xavier/Python/blob/main/Taxi_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Importação das bibliotecas necessárias
import numpy as np  # Para manipulações de arrays e cálculos matemáticos
import gym  # Para carregar o ambiente Taxi-v3
import pygame  # Para criar uma visualização interativa
import time  # Para controle do intervalo entre frames na visualização

# Configuração do ambiente Taxi-v3
env = gym.make("Taxi-v3")  # Inicializa o ambiente Taxi-v3 do Gym

# Inicializa a tabela Q com zeros
# Dimensão: número de estados x número de ações possíveis
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hiperparâmetros para o treinamento
alpha = 0.1  # Taxa de aprendizado
gamma = 0.6  # Fator de desconto (importância de recompensas futuras)
epsilon = 0.1  # Taxa de exploração (probabilidade de ações aleatórias)
num_episodes = 1000  # Número total de episódios para treinamento

# Configurações do Pygame para visualização
pygame.init()  # Inicializa o Pygame
screen_size = 500  # Dimensão da janela (500x500 pixels)
cell_size = screen_size // 5  # Cada célula na grade tem 100x100 pixels
screen = pygame.display.set_mode((screen_size, screen_size))  # Cria a janela do Pygame
pygame.display.set_caption("Taxi-v3 Visualization")  # Título da janela do Pygame

# Define as cores para os elementos visuais
colors = {
    "taxi": (255, 255, 0),  # Táxi: amarelo
    "passenger": (0, 255, 0),  # Passageiro: verde
    "destination": (255, 0, 0),  # Destino: vermelho
    "road": (200, 200, 200),  # Estrada: cinza
}

# Função para renderizar o ambiente no Pygame
def render_env(state):
    """
    Renderiza o estado atual do ambiente Taxi-v3 na janela do Pygame.
    :param state: Estado atual do ambiente (inteiro).
    """
    screen.fill(colors["road"])  # Preenche o fundo com a cor da estrada

    # Decodifica o estado para obter posições do táxi, passageiro e destino
    taxi_row, taxi_col, passenger, destination = env.unwrapped.decode(state)

    # Desenha a grade 5x5
    for i in range(5):
        for j in range(5):
            pygame.draw.rect(screen, (0, 0, 0), (j * cell_size, i * cell_size, cell_size, cell_size), 1)

    # Desenha o táxi na posição atual
    pygame.draw.rect(screen, colors["taxi"],
                     (taxi_col * cell_size + 10, taxi_row * cell_size + 10, cell_size - 20, cell_size - 20))

    # Desenha o passageiro (se ainda não foi pego)
    if passenger < 4:  # Passageiro está em uma das localizações fixas
        passenger_coords = [(0, 0), (0, 4), (4, 0), (4, 3)]  # Locais fixos no ambiente
        passenger_row, passenger_col = passenger_coords[passenger]
        pygame.draw.circle(screen, colors["passenger"],
                           (passenger_col * cell_size + cell_size // 2, passenger_row * cell_size + cell_size // 2), 10)

    # Desenha o destino
    destination_coords = [(0, 0), (0, 4), (4, 0), (4, 3)]  # Locais fixos de destino
    destination_row, destination_col = destination_coords[destination]
    pygame.draw.circle(screen, colors["destination"],
                       (destination_col * cell_size + cell_size // 2, destination_row * cell_size + cell_size // 2), 10)

    pygame.display.flip()  # Atualiza a janela com o novo estado

# Treinamento do agente com Q-Learning
for episode in range(num_episodes):
    reset_result = env.reset()  # Reseta o ambiente para o início do episódio
    # Para versões recentes do Gym, verifica o tipo do retorno
    state = reset_result if isinstance(reset_result, int) else reset_result[0]
    done = False  # Define o episódio como não terminado

    while not done:
        # Escolha da ação: Exploração (aleatória) ou Exploração (melhor ação)
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Escolha aleatória
        else:
            action = np.argmax(q_table[state])  # Melhor ação com base na tabela Q

        # Realiza a ação escolhida e obtém o próximo estado, recompensa e estado de término
        step_result = env.step(action)
        # Para versões recentes do Gym, verifica o tipo do retorno
        next_state = step_result[0] if isinstance(step_result, tuple) else step_result
        reward = step_result[1]  # Recompensa obtida pela ação
        done = step_result[2]  # Verifica se o episódio terminou

        # Atualiza a tabela Q usando a fórmula do Q-Learning
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])  # Melhor estimativa para o próximo estado
        q_table[state, action] = old_value + alpha * (reward + gamma * next_max - old_value)

        # Atualiza o estado atual para o próximo estado
        state = next_state

# Teste do agente treinado com visualização
reset_result = env.reset()  # Reseta o ambiente para o início do teste
state = reset_result if isinstance(reset_result, int) else reset_result[0]
done = False  # Define o episódio como não terminado

while not done:
    for event in pygame.event.get():  # Verifica eventos do Pygame (como fechar a janela)
        if event.type == pygame.QUIT:
            pygame.quit()
            quit()

    # Escolhe a melhor ação com base na tabela Q treinada
    action = np.argmax(q_table[state])

    # Executa a ação e avança para o próximo estado
    step_result = env.step(action)
    state = step_result[0] if isinstance(step_result, tuple) else step_result
    done = step_result[2]  # Verifica se o episódio terminou

    # Renderiza o ambiente no Pygame
    render_env(state)
    time.sleep(0.5)  # Pausa para visualização mais lenta

pygame.quit()  # Encerra o Pygame
